In [1]:
import requests

from bs4 import BeautifulSoup as bs

import pandas as pd
pd.set_option('display.max_columns', None)

import multiprocessing as mp

from tqdm.notebook import tqdm

import time

from joblib import Parallel, delayed

In [2]:
anime = pd.read_csv('anime_2.csv')

In [3]:
def wiki_dir(title): # Entra una lista de titulos
    
    res = 'unknown'
    
    url_wiki = 'https://en.wikipedia.org/wiki/'
    
    url= url_wiki + title.replace(' ','_')
   
    # Intenta encontrar el link y parsearlo

    try:
        
        html = requests.get(url).content
        
        soup = bs(html, "html.parser")
        
        # Prueba class_='infobox'
        
        if str(soup.find('table', class_='infobox')) != 'None':
                                                                       
            sauce = soup.find('table', class_='infobox')

            noodle = sauce.find_all('tr')
            
        # Prueba class_='infobox infobox_v2'
            
        elif str(soup.find('table', class_='infobox infobox_v2')) != 'None': 
            
            sauce = soup.find('table', class_='infobox infobox_v2')

            noodle = sauce.find_all('tr')
        
        # No encuentra la tabla que buscamos
        
        else: 
            
            pass
        
        # Loop que busca el th 'directed by' en todas las filas de la tabla y devuelve el nombre del director.
        
        for row in noodle:

            if row.find('th') == None:
                
                continue
                
            elif row.find('th').text.lower().replace('\xa0','') == 'producedby':
                
                res = row.find('td').text
            
            else:
                
                continue
        
        # Si se completa el loop sin haber conseguido un director devuelve esto.
                
        return res
        
    except:
        
        return 'fallo'

In [4]:

paralelo = Parallel(n_jobs=4,  verbose=True)


anime['producer'] = paralelo(delayed(wiki_dir)(title) for title in anime.english_name)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   21.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   46.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:  5.9min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed:  7.2min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed:  8.5min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 10.0min
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed: 11.4min
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed: 13.0min
[Parallel(n_jobs=4)]: Done 11242 tasks      | elapsed: 14.9mi

In [ ]:
anime.to_csv('../data/anime_3.csv', index = False)